In [0]:
import torch.nn as nn
import torch

import torch.optim as optim

import pandas as pd
import numpy as np



In [0]:
xt=pd.read_csv("m_train3b.csv")
xt['clicked'].shape

(26784,)

In [2]:
xt=pd.read_csv("m_train3b.csv")
y_train=xt['clicked']
x_train=xt.drop(['clicked'],axis=1)

xt2=pd.read_csv("m_test3b.csv")
y_test=xt2['clicked']
x_test=xt2.drop(['clicked'],axis=1)
#print(x_test.shape)
#print(x_train.shape)

x2=x_test.copy()
y2=y_test.copy()

print(x_test.shape)

x_valid=x2.iloc[:3350,:]
y_valid=y2.iloc[:3350,]
#print(x_valid.shape)
#print(y_valid.shape)

x_test=x_test.iloc[3351:,:]
y_test=y_test.iloc[3351:,]
#print(x_test.shape)
#print(y_test.shape)

x_train.fillna(0,inplace=True)
x_test.fillna(0,inplace=True)
x_valid.fillna(0,inplace=True)

#x_test.fillna(0,inplace=True)
#print(sum(x_train.isna().any()))
#print(sum(x_test.isna().any()))
#print(sum(x_valid.isna().any()))

seq_length=8
batch_size=70
x1=torch.ones([x_train.shape[0]-seq_length+1,seq_length,x_train.shape[1]])
y1=torch.ones([(x_train.shape[0]-seq_length+1),seq_length])


y_valid.shape

(6748, 2200)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


(3350,)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
#Training Model

x1=torch.ones([x_train.shape[0]-seq_length+1,seq_length,x_train.shape[1]])
y1=torch.ones([(x_train.shape[0]-seq_length+1),seq_length])

for i in range(x_train.shape[0]-seq_length+1):
    x1[i] =  torch.tensor(x_train.iloc[i:i+seq_length, :].values)
    y1[i] = torch.tensor(y_train.iloc[i:i+seq_length].values)

#x1.shape
y1 = y1[:, :]

order = torch.randperm(len(x1))
x1 = x1[order]
y1 = y1[order]

print(x1.shape)
print(y1.shape)

torch.Size([26777, 8, 2200])
torch.Size([26777, 8])


In [21]:
#testing Model

x2=torch.ones([x_test.shape[0]-seq_length+1,seq_length,x_test.shape[1]])
y2=torch.ones([(x_test.shape[0]-seq_length+1),seq_length])


for i in range(x_test.shape[0]-seq_length+1):
    x2[i] =  torch.tensor(x_test.iloc[i:i+seq_length, :].values)
    y2[i] = torch.tensor(y_test.iloc[i:i+seq_length].values)

#x2.shape
y2 = y2[:, : ]

print(x2.shape)
print(y2.shape)

torch.Size([3390, 8, 2200])
torch.Size([3390, 8])


In [22]:
#validing Model

xv=torch.ones([x_valid.shape[0]-seq_length+1,seq_length,x_valid.shape[1]])
yv=torch.ones([(x_valid.shape[0]-seq_length+1),seq_length])


for i in range(x_valid.shape[0]-seq_length+1):
    xv[i] =  torch.tensor(x_valid.iloc[i:i+seq_length, :].values)
    yv[i] = torch.tensor(y_valid.iloc[i:i+seq_length].values)

#xv.shape
yv = yv[:, : ]

print(xv.shape)
print(yv.shape)

torch.Size([3343, 8, 2200])
torch.Size([3343, 8])


In [0]:
#Vanilla LSTM
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.lstm = nn.LSTM(2200,32) #defining features and hidden dimensions
        self.linear= nn.Linear(32, 2)
       

    def forward(self, x):
        out, (h, c) = self.lstm(x) # h,c are the last hidden and cell state; out has all the hidden states 
         #taking only the last hidden state for input
        
        x=self.linear(out)
   
        return x

In [0]:
#Define criterions like loss function and optimizer

net = MyModule()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=1e-3, weight_decay=1e-4)




In [27]:
x1 = torch.transpose(x1, 0, 1)
y1 = torch.transpose(y1, 0, 1)
n_epoch=100
xv = torch.transpose(xv, 0, 1)
yv = torch.transpose(yv, 0, 1)
steps = x1.shape[1] // batch_size
valid_err_fin = 1




for epoch in range(n_epoch):
    print(valid_err_fin)
    train_error = 0.
    for step in range(steps):
        X = x1[:, step * batch_size : (step + 1) * batch_size, :]
        y = y1[:, step * batch_size : (step + 1) * batch_size]
        y = y.reshape(-1).long()
        
        y_pred = net(X)
       
        #y_pred = y_pred.repeat(1, 1, 2)
        #y_pred[:, :, 1] = 1 - y_pred[:, :, 0]
        y_pred = y_pred.view(-1, 2)
        
        train_loss = criterion(y_pred, y)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        train_error += train_loss.item()

        
    
    step2s = xv.shape[1] // batch_size
    valid_err=0.

    miss_class=0
    for step2 in range(step2s):

        


        Xv = xv[:, step2 * batch_size : (step2 + 1) * batch_size, :]
        yvalid = yv[:, step2 * batch_size : (step2 + 1) * batch_size]
        yvalid = yvalid.reshape(-1).long()
    
        y_pred2 = net(Xv)
       
    #y_pred = y_pred.repeat(1, 1, 2)
    #y_pred[:, :, 1] = 1 - y_pred[:, :, 0]
        y_pred2 = y_pred2.view(-1, 2)

        y_pred2 = torch.argmax(y_pred2, dim=-1)
        
        valid_err+=sum(y_pred2!=yvalid)/(float(len(yvalid)))
        

        #valid_loss = criterion(y_pred2, y2)
    #optimizer.zero_grad()
    #train_loss.backward()
    #optimizer.step2()
    
        #valid_error+=valid_loss.item()

    valid_error=valid_err/step2s

    if(valid_error<valid_err_fin):
        valid_err_fin=valid_error
        torch.save(net.state_dict(), 'checkpoint.pth')
    print("Epoch: {}, Training Error, {}, Valid Error, {}".format(epoch, train_error/steps,valid_error))    
    torch.save(net.state_dict(), 'checkpoint.pth') 

    #for g in optimizer.param_groups:
     #   g['lr'] *= 0.95
      #  print("LR: {}".format(g['lr']))  
    


1
Epoch: 0, Training Error, 0.4064754871797811, Valid Error, 0.16208207607269287
tensor(0.1621)
Epoch: 1, Training Error, 0.309187545436215, Valid Error, 0.1613221913576126
tensor(0.1613)
Epoch: 2, Training Error, 0.2508412299980044, Valid Error, 0.16812309622764587
tensor(0.1613)
Epoch: 3, Training Error, 0.19154431865945537, Valid Error, 0.17670972645282745
tensor(0.1613)
Epoch: 4, Training Error, 0.14314001433902385, Valid Error, 0.1828647404909134
tensor(0.1613)
Epoch: 5, Training Error, 0.11030750103411874, Valid Error, 0.18795590102672577
tensor(0.1613)
Epoch: 6, Training Error, 0.08856499263363359, Valid Error, 0.19053947925567627
tensor(0.1613)
Epoch: 7, Training Error, 0.07357714616776449, Valid Error, 0.19308508932590485
tensor(0.1613)
Epoch: 8, Training Error, 0.06274390898410874, Valid Error, 0.19335106015205383
tensor(0.1613)
Epoch: 9, Training Error, 0.05459712554959102, Valid Error, 0.19460487365722656
tensor(0.1613)
Epoch: 10, Training Error, 0.04833443638385902, Valid 

KeyboardInterrupt: ignored

In [28]:
net.load_state_dict(torch.load('checkpoint.pth'))


#testing Model

x2=torch.ones([x_test.shape[0]-seq_length+1,seq_length,x_test.shape[1]])
y2=torch.ones([(x_test.shape[0]-seq_length+1),seq_length])


for i in range(x_test.shape[0]-seq_length+1):
    x2[i] =  torch.tensor(x_test.iloc[i:i+seq_length, :].values)
    y2[i] = torch.tensor(y_test.iloc[i:i+seq_length].values)

#x2.shape
y2 = y2[:, : ]
x2 = torch.transpose(x2, 0, 1)
y2 = torch.transpose(y2, 0, 1)



step3s = x2.shape[1] // batch_size
valid_err2=0.



for step3 in range(step3s):
   


    X3 = x2[:, step3 * batch_size : (step3 + 1) * batch_size, :]
    y3 = y2[:, step3 * batch_size : (step3 + 1) * batch_size]
    y3 = y3.reshape(-1).long()
    
    y_pred3 = net(X3)
       
    #y_pred = y_pred.repeat(1, 1, 2)
    #y_pred[:, :, 1] = 1 - y_pred[:, :, 0]
    y_pred3 = y_pred3.view(-1, 2)

    y_pred3 = torch.argmax(y_pred3, dim=-1)
        
    valid_err2+=sum(y_pred3!=y3)/(float(len(y3)))
        

        #valid_loss = criterion(y_pred2, y2)
    #optimizer.zero_grad()
    #train_loss.backward()
    #optimizer.step2()
    print(step3)
        #valid_error+=valid_loss.item()

valid_error2=valid_err2/step3s

print(valid_error2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
tensor(0.2034)


In [0]:
sum(y_pred2!=y2)/1120.

RuntimeError: ignored

In [1]:


class BLSTM_Cell(nn.Module): # new LSTM cell
    def __init__(self, in_features=2200, hidden_features=4):
        super(BLSTM_Cell, self).__init__()
        
        self.in_features = in_features #features dim
        self.hidden_features = hidden_features #hidden dim
        
        self.Wxh = nn.Linear(in_features+hidden_features-90, 4*hidden_features)
        self.Wxd = nn.Linear(in_features-60+27, hidden_features)
        self.Wxa = nn.Linear(in_features-60+63, hidden_features)
        
    def forward(self, x, h=None, c=None):
        if h is None:
            h = torch.zeros([x.shape[0], self.hidden_features])
        if c is None:
            c = torch.zeros([x.shape[0], self.hidden_features])         
            
        xh = torch.cat([x[:, :-90], h], dim=-1)
        xd = x[:, :-33] 
        xa = torch.cat([x[:, :-90], x[:, -63:]], dim=-1) 
        
        xh = self.Wxh(xh)
        xd = self.Wxd(xd)
        xa = self.Wxa(xa)
        
        i = torch.sigmoid(xh[:, :self.hidden_features])
        f = torch.sigmoid(xh[:, self.hidden_features:2*self.hidden_features])
        t = torch.sigmoid(xd)
        a = torch.sigmoid(xa)
        ci = torch.tanh(xh[:, 2*self.hidden_features:3*self.hidden_features])
        c = f * c + i * a * t * ci
        o = torch.sigmoid(xh[:, 3*self.hidden_features:])
        h = o * torch.tanh(c)
        
        return h, c
    
cell = BLSTM_Cell()
#inputs = torch.randn([4, 6])
#h, c = cell(inputs)

class BLSTM(nn.Module):
    def __init__(self, in_features=2200, hidden_features=4):
        super(BLSTM, self).__init__()
        
        self.cell = BLSTM_Cell(in_features=in_features, hidden_features=hidden_features)
        
    def forward(self, x, h0=None, c0=None):
        seq_len = len(x)
        out = []
        for n in range(seq_len):
            if n == 0:
                h, c = self.cell(x[n], h0, c0)
            else:
                h, c = self.cell(x[n], h, c)
            out.append(h)
        out = torch.stack(out)
        
        return out, (h, c)   

#blstm = BLSTM(in_features=1830)

#out,(h,c)=blstm(x1)
#c.shape

NameError: ignored

In [0]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.lstm = BLSTM(in_features=2200,hidden_features=8 ) #defining features and hidden dimensions
        self.linear= nn.Linear(8, 2)
       

    def forward(self, x):
        out, (h, c) = self.lstm(x) # h,c are the last hidden and cell state; out has all the hidden states 
         #taking only the last hidden state for input
        
        x=self.linear(out)
   
        return x

In [0]:
#Define criterions like loss function and optimizer

net = MyModule()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=1e-3, weight_decay=1e-4)




In [32]:
#x1 = torch.transpose(x1, 0, 1)
#y1 = torch.transpose(y1, 0, 1)
n_epoch=100
#xv = torch.transpose(xv, 0, 1)
#yv = torch.transpose(yv, 0, 1)
steps = x1.shape[1] // batch_size
valid_err_fin = 1


for epoch in range(n_epoch):
    print(valid_err_fin)
    train_error = 0.
    for step in range(steps):
        X = x1[:, step * batch_size : (step + 1) * batch_size, :]
        y = y1[:, step * batch_size : (step + 1) * batch_size]
        y = y.reshape(-1).long()
        
        y_pred = net(X)
       
        #y_pred = y_pred.repeat(1, 1, 2)
        #y_pred[:, :, 1] = 1 - y_pred[:, :, 0]
        y_pred = y_pred.view(-1, 2)
        
        train_loss = criterion(y_pred, y)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        train_error += train_loss.item()

        
    
    step2s = xv.shape[1] // batch_size
    valid_err=0.

    miss_class=0
    for step2 in range(step2s):

        


        Xv = xv[:, step2 * batch_size : (step2 + 1) * batch_size, :]
        yvalid = yv[:, step2 * batch_size : (step2 + 1) * batch_size]
        yvalid = yvalid.reshape(-1).long()
    
        y_pred2 = net(Xv)
       
    #y_pred = y_pred.repeat(1, 1, 2)
    #y_pred[:, :, 1] = 1 - y_pred[:, :, 0]
        y_pred2 = y_pred2.view(-1, 2)

        y_pred2 = torch.argmax(y_pred2, dim=-1)
        
        valid_err+=sum(y_pred2!=yvalid)/(float(len(yvalid)))
        

        #valid_loss = criterion(y_pred2, y2)
    #optimizer.zero_grad()
    #train_loss.backward()
    #optimizer.step2()
    
        #valid_error+=valid_loss.item()

    valid_error=valid_err/step2s

    if(valid_error<valid_err_fin):
        valid_err_fin=valid_error
        torch.save(net.state_dict(), 'checkpoint2.pth')
    print("Epoch: {}, Training Error, {}, Valid Error, {}".format(epoch, train_error/steps,valid_error))    
    torch.save(net.state_dict(), 'checkpoint2.pth') 

    for g in optimizer.param_groups:
        g['lr'] *= 0.95
        print("LR: {}".format(g['lr']))  
    


1
Epoch: 0, Training Error, 0.4669766277854979, Valid Error, 0.16371579468250275
LR: 0.00095
tensor(0.1637)
Epoch: 1, Training Error, 0.3904727639640189, Valid Error, 0.16337384283542633
LR: 0.0009025
tensor(0.1634)
Epoch: 2, Training Error, 0.3350461964051761, Valid Error, 0.15664894878864288
LR: 0.000857375
tensor(0.1566)
Epoch: 3, Training Error, 0.3098528805671562, Valid Error, 0.15661095082759857
LR: 0.0008145062499999999
tensor(0.1566)
Epoch: 4, Training Error, 0.29220307801718487, Valid Error, 0.15908055007457733
LR: 0.0007737809374999998
tensor(0.1566)
Epoch: 5, Training Error, 0.2788712754845619, Valid Error, 0.16287992894649506
LR: 0.0007350918906249997
tensor(0.1566)
Epoch: 6, Training Error, 0.26756067819776336, Valid Error, 0.1659574657678604
LR: 0.0006983372960937497
tensor(0.1566)
Epoch: 7, Training Error, 0.2569049034287168, Valid Error, 0.1688070148229599
LR: 0.0006634204312890621
tensor(0.1566)
Epoch: 8, Training Error, 0.2465660987100052, Valid Error, 0.1714665442705

KeyboardInterrupt: ignored

In [33]:
net.load_state_dict(torch.load('checkpoint2.pth'))


#testing Model

x2=torch.ones([x_test.shape[0]-seq_length+1,seq_length,x_test.shape[1]])
y2=torch.ones([(x_test.shape[0]-seq_length+1),seq_length])


for i in range(x_test.shape[0]-seq_length+1):
    x2[i] =  torch.tensor(x_test.iloc[i:i+seq_length, :].values)
    y2[i] = torch.tensor(y_test.iloc[i:i+seq_length].values)

#x2.shape
y2 = y2[:, : ]
x2 = torch.transpose(x2, 0, 1)
y2 = torch.transpose(y2, 0, 1)



step3s = x2.shape[1] // batch_size
valid_err2=0.



for step3 in range(step3s):
   


    X3 = x2[:, step3 * batch_size : (step3 + 1) * batch_size, :]
    y3 = y2[:, step3 * batch_size : (step3 + 1) * batch_size]
    y3 = y3.reshape(-1).long()
    
    y_pred3 = net(X3)
       
    #y_pred = y_pred.repeat(1, 1, 2)
    #y_pred[:, :, 1] = 1 - y_pred[:, :, 0]
    y_pred3 = y_pred3.view(-1, 2)

    y_pred3 = torch.argmax(y_pred3, dim=-1)
        
    valid_err2+=sum(y_pred3!=y3)/(float(len(y3)))
        

        #valid_loss = criterion(y_pred2, y2)
    #optimizer.zero_grad()
    #train_loss.backward()
    #optimizer.step2()
    print(step3)
        #valid_error+=valid_loss.item()

valid_error2=valid_err2/step3s

print(valid_error2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
tensor(0.1756)
